In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys; sys.path.append("/data/jerrylee/pjt/BIGFAM.v.2.0")
from src import frreg, tools
import importlib
import scipy.stats as stats
import statsmodels.formula.api as smf

In [2]:
cohort = "UKB"

# Step 1. Load relatives phenotype data

In [3]:
# parameters
info_fn = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/relationship_information/relatives.formatted.info"

In [4]:
# relative information format
df_pair = pd.read_csv(info_fn, sep='\t')
df_pair.head()

,DOR,rcode,relationship,volid,relid,volage,relage,volsex,relsex,Erx
0,1,SB,daughter-sister,1000094,3653174,65,64,F,F,0.75
1,1,NaN,NaN,1000220,1691267,64,64,F,F,NaN
2,1,NaN,NaN,1000286,1571411,53,70,F,F,NaN
3,1,NaN,NaN,1000295,1045127,60,41,F,F,NaN
4,1,NaN,NaN,1000476,3599303,50,51,F,M,NaN


# Step 2. Do FR-reg

In [5]:
pheno_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/phenotype"
frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/test"

In [6]:
pheno_fns = os.listdir(pheno_path)
len(pheno_fns)

106

## Step 2.1. DOR level

In [7]:
thred_pair=100
n_bootstrap=100
std_pheno=True 
remove_outlier=True

In [8]:
def bootstrap_regression(
    df_dor, 
    reg_type="intercept", # intercept or no_intercept
    n_bootstrap=100, 
    progress=True
):
    slopes = np.zeros(n_bootstrap)
    
    # volid별로 인덱스를 미리 준비
    unique_volids = df_dor['volid'].unique()
    vol_to_indices = {vol: df_dor.index[df_dor['volid'] == vol].tolist() 
                     for vol in unique_volids}
    
    if progress:
        iterator = tqdm(range(n_bootstrap))
    else:
        iterator = range(n_bootstrap)
    
    for i in iterator:
        # 각 volid마다 하나의 인덱스만 랜덤 선택
        selected_indices = np.array([np.random.choice(indices) 
                                   for indices in vol_to_indices.values()])
        
        # 첫 번째 필터링된 데이터
        df_filtered = df_dor.iloc[selected_indices]
        
        # 두 번째 resampling
        df_resampled = df_filtered.sample(n=len(df_dor), replace=True)
        
        # statsmodels를 사용한 회귀분석
        if reg_type == "no_intercept":
            model = smf.ols('volpheno ~ 0 + relpheno', data=df_resampled)
        else:
            model = smf.ols('volpheno ~ relpheno', data=df_resampled)
        results = model.fit()
        slopes[i] = results.params['relpheno']
    
    slope_estimate = np.mean(slopes)
    slope_se = np.std(slopes)
    t_stat = slope_estimate / slope_se
    p_value = 2 * (1 - stats.t.cdf(abs(t_stat), len(df_dor)-1))
    
    return {
        'slope': slope_estimate,
        'se': slope_se,
        'p': p_value,
        'n': len(df_dor)
    }

In [67]:
def std_col(vals):
    return (vals - np.mean(vals)) / np.std(vals)

for fn in tqdm(pheno_fns):
    pheno = fn.split(".")[0]
    pheno_fn = f"{pheno_path}/{fn}"
    df_pheno = pd.read_csv(pheno_fn, sep="\t")
    df_pheno = frreg.remove_outliers(df_pheno, "pheno")
    
    if std_pheno:
        df_pheno["pheno"] = std_col(df_pheno["pheno"])
    
    # merge pheno with relatives
    df_mrg = frreg.merge_pheno_info(df_pheno, df_pair)
    
    # unique id set
    df = tools.remove_duplicate_relpair(df_mrg, ["volid", "relid"])
    
    # flip and concat volid and relid
    colns_to_flip = {"volid": "relid", 
                     "volage": "relage", 
                     "volsex": "relsex",
                     "volpheno": "relpheno"}
    df = (tools.flip_and_concat(df.copy(), colns_to_flip)
          .reset_index(drop=True))
    
    # regression by each DOR
    df_no_intercept = pd.DataFrame(columns=["DOR", "slope", "se", "p", "n"])
    df_intercept = pd.DataFrame(columns=["DOR", "slope", "se", "p", "n"])
    
    for d in sorted(df["DOR"].unique()):
        df_dor = df[df["DOR"] == d].copy().reset_index(drop=True)
        
        if len(df_dor) < thred_pair: # < 5 relative pairs
            raise Exception("Too small pairs..")
        
        # with intercept
        results_no_intercept = bootstrap_regression(
            df_dor, 
            reg_type="no_intercept",
            n_bootstrap=n_bootstrap,
            progress=True
        )
        
        tmp_no_intercept = pd.DataFrame(results_no_intercept, index=[0])
        tmp_no_intercept["DOR"] = d
        df_no_intercept = (pd.concat([df_no_intercept, tmp_no_intercept])
                           .reset_index(drop=True))
    
        # without intercept
        results_intercept = bootstrap_regression(
            df_dor, 
            reg_type="intercept",
            n_bootstrap=n_bootstrap,
            progress=True
        )
        tmp_intercept = pd.DataFrame(results_intercept, index=[0])
        tmp_intercept["DOR"] = d
        df_intercept = (pd.concat([df_intercept, tmp_intercept])
                        .reset_index(drop=True))    
    
    df_no_intercept.to_csv(
            f"{frreg_path}/{pheno}.no_intercept.frreg",
            sep='\t',
            index=False
        )

    df_intercept.to_csv(
            f"{frreg_path}/{pheno}.intercept.frreg",
            sep='\t',
            index=False
        )


  0%|          | 0/106 [02:42<?, ?it/s]


## Step 2.2. REL level

In [9]:
def std_col(vals):
    return (vals - np.mean(vals)) / np.std(vals)

for fn in tqdm(pheno_fns):
    pheno = fn.split(".")[0]
    pheno_fn = f"{pheno_path}/{fn}"
    df_pheno = pd.read_csv(pheno_fn, sep="\t", names=["eid", "iid", "pheno"])
    df_pheno = frreg.remove_outliers(df_pheno[["eid", "pheno"]], "pheno")
    
    if std_pheno:
        df_pheno["pheno"] = std_col(df_pheno["pheno"])
    
    # merge pheno with relatives
    df_mrg = frreg.merge_pheno_info(df_pheno, df_pair)
    
    # unique id set
    df = tools.remove_duplicate_relpair(df_mrg, ["volid", "relid"])
    
    # flip and concat volid and relid
    colns_to_flip = {"volid": "relid", 
                     "volage": "relage", 
                     "volsex": "relsex",
                     "volpheno": "relpheno"}
    df = (tools.flip_and_concat(df.copy(), colns_to_flip)
          .reset_index(drop=True))
    
    # regression by each DOR
    df_intercept = pd.DataFrame(columns=["DOR", "relationship", "slope", "se", "p", "n"])
    
    for rel in df["relationship"].unique():
        df_rel = df[df["relationship"] == rel].copy().reset_index(drop=True)
        
        if len(df_rel) < thred_pair: # < 100 relative pairs
            continue
        
        results_intercept = bootstrap_regression(
            df_rel, 
            reg_type="intercept",
            n_bootstrap=n_bootstrap,
            progress=False
        )
        tmp_intercept = pd.DataFrame(results_intercept, index=[0])
        tmp_intercept["DOR"] = df_rel["DOR"].unique()[0]
        tmp_intercept["relationship"] = rel

        df_intercept = (pd.concat([df_intercept, tmp_intercept])
                        .reset_index(drop=True))    
        
    df_intercept.to_csv(
            f"{frreg_path}/{cohort}.{pheno}.REL.frreg",
            sep='\t',
            index=False
        )
    break


  0%|          | 0/106 [00:00<?, ?it/s]/tmp/ipykernel_237234/2111081117.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_intercept = (pd.concat([df_intercept, tmp_intercept])
  0%|          | 0/106 [01:15<?, ?it/s]


# Step 3. PO-SIB


In [10]:
output_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/po-sib"
output_path

'/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/po-sib'

In [13]:
subgroups = ["PC", "SB"]

for fn in tqdm(pheno_fns):
    pheno = fn.split(".")[0]
    pheno_fn = f"{pheno_path}/{fn}"
    df_pheno = pd.read_csv(pheno_fn, sep="\t")
    df_pheno = frreg.remove_outliers(df_pheno, "pheno")
    
    for subgroup in subgroups:
        # DOR=1인 데이터 중 rcode="SB"만 남기고, 나머지 DOR은 유지
        df_pair_filtered = pd.concat([
            # DOR=1이면서 rcode="SB"인 데이터
            df_pair[(df_pair["DOR"] == 1) & (df_pair["rcode"] == subgroup)],
            # DOR이 1이 아닌 모든 데이터
            df_pair[df_pair["DOR"] != 1]
        ]).reset_index(drop=True)

        # merge pheno with relatives
        df_mrg = frreg.merge_pheno_info(df_pheno, df_pair_filtered)
        break
        try:
            df_frreg, msgs = frreg.familial_relationship_regression_DOR(
                df_mrg.drop(columns=["rcode", "relationship", "Erx"]),
                n_bootstrap=100
            )
            
            df_frreg.to_csv(
                f"{output_path}/{pheno}.{subgroup}.frreg",
                sep='\t',
                index=False
            )
                
        except:
            continue
    break

  0%|          | 0/40 [00:00<?, ?it/s]


In [17]:
df_pair.loc[df_pair["DOR"] == 1]["rcode"].unique()

array(['SB', 'PC'], dtype=object)

In [27]:
df_frreg

,DOR,slope,se,p,n
0,1,0.189230,0.008803,0.0,8724
1,2,0.092695,0.007556,0.0,17450
2,3,0.045820,0.003846,0.0,82132
